In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys; sys.path.insert(0, '..') # add parent folder path where lib folder is

In [3]:
# from ml.preprocess import data
import pickle
from dask_ml.preprocessing import StandardScaler
from dask_ml.model_selection import train_test_split

from sklearn.metrics import r2_score, mean_squared_error
from sklearn.metrics import f1_score, roc_auc_score, precision_score, recall_score

import numpy as np
import pandas as pd

from utils import helper, config, dasker

from ml.models.base.slug_ann import SlugANN
from ml.models.base.slug_xgboost import SlugXGBoost
from ml.models.base.brisk_xgboost import BriskXGBoost
from ml.models.ensemble import BaseUnify
from ml.models.autofit import AutoFit

from ml.xai.model.explainable import Explainable


2022-10-26 19:48:22,330 - matplotlib - DEBUG - matplotlib data path: /home/vagrant/miniconda3/envs/test/lib/python3.8/site-packages/matplotlib/mpl-data
2022-10-26 19:48:22,338 - matplotlib - DEBUG - CONFIGDIR=/home/vagrant/.config/matplotlib
2022-10-26 19:48:22,341 - matplotlib - DEBUG - interactive is False
2022-10-26 19:48:22,343 - matplotlib - DEBUG - platform is linux
2022-10-26 19:48:22,427 - matplotlib - DEBUG - CACHEDIR=/home/vagrant/.cache/matplotlib
2022-10-26 19:48:22,431 - matplotlib.font_manager - DEBUG - Using fontManager instance from /home/vagrant/.cache/matplotlib/fontlist-v330.json


In [4]:
X, y = helper.get_covid_dataset()
# X = X.drop(['location'], axis = 1)
# df_X_scalar = StandardScaler().fit_transform(X)
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)

In [5]:
client = dasker.get_local_client()

2022-10-26 19:48:23,929 - asyncio - DEBUG - Using selector: EpollSelector


2022-10-26 19:48:25,502 - distributed.diskutils - INFO - Found stale lock file and directory '/tmp/dask-worker-space/worker-ih12liv9', purging
2022-10-26 19:48:25,503 - distributed.diskutils - INFO - Found stale lock file and directory '/tmp/dask-worker-space/worker-aocxzolw', purging
2022-10-26 19:48:25,503 - distributed.diskutils - INFO - Found stale lock file and directory '/tmp/dask-worker-space/worker-kgp6k_w0', purging
2022-10-26 19:48:25,503 - distributed.diskutils - INFO - Found stale lock file and directory '/tmp/dask-worker-space/worker-_bu0s81r', purging


In [6]:
### dont forget the '/' in the path
config.project_name = '/covid_xai'

In [7]:
auto_learn = AutoFit(df_X=X, df_y=y, pred_class='regression', retrain=True)

In [8]:
auto_learn.fetch_models()

Cleared the existing saved models
Directory structure for project /covid_autolearn_v2 created successfully
2022-10-26 19:48:29,731 - console_info - INFO - ensemble: base models discovery started
2022-10-26 19:48:29,731 - console_info - INFO - ensemble: base models discovery started
2022-10-26 19:48:32,064 - matplotlib - DEBUG - matplotlib data path: /home/vagrant/miniconda3/envs/test/lib/python3.8/site-packages/matplotlib/mpl-data
2022-10-26 19:48:32,071 - matplotlib - DEBUG - CONFIGDIR=/home/vagrant/.config/matplotlib
2022-10-26 19:48:32,073 - matplotlib - DEBUG - interactive is False
2022-10-26 19:48:32,074 - matplotlib - DEBUG - platform is linux
2022-10-26 19:48:32,087 - matplotlib - DEBUG - matplotlib data path: /home/vagrant/miniconda3/envs/test/lib/python3.8/site-packages/matplotlib/mpl-data
2022-10-26 19:48:32,094 - matplotlib - DEBUG - CONFIGDIR=/home/vagrant/.config/matplotlib
2022-10-26 19:48:32,097 - matplotlib - DEBUG - interactive is False
2022-10-26 19:48:32,098 - matplo

In [9]:
auto_learn.base_unify.scores

[[-0.14227871176631646, -2.9533921810424606],
 [0.8577262465705702, -0.7424269662306409]]

In [10]:
models = []
for base_model in auto_learn.base_unify.base_models:
    models.append(base_model.best_fit)

In [11]:
xai = Explainable(df_X=X, df_y=y, models=models)

In [12]:
attr_algos = ['IG', 'SHAP', 'GradientSHAP']
res = xai.get_attr(attr_algos=['IG', 'SHAP', 'GradientSHAP'])

calculating variable importance on Sequential(
  (0): Linear(in_features=41, out_features=25, bias=True)
  (1): ReLU()
  (2): Dropout(p=0.38472422688708374, inplace=False)
  (3): Linear(in_features=25, out_features=222, bias=True)
  (4): ReLU()
  (5): Dropout(p=0.38472422688708374, inplace=False)
  (6): Linear(in_features=222, out_features=229, bias=True)
  (7): ReLU()
  (8): Dropout(p=0.38472422688708374, inplace=False)
  (9): Linear(in_features=229, out_features=1, bias=True)
  (10): ReLU()
)
calculating variable importance on <xgboost.core.Booster object at 0x7f13d04b9340>


Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
[09:07:33] WARNING: /mnt/c/Users/rwmas/xgboost/src/c_api/c_api.cc:967: Saving into deprecated binary model format, please consider using `json` or `ubj`. Model format will default to JSON in XGBoost 2.2 if not specified.
[09:07:33] WARNING: /mnt/c/Users/rwmas/xgboost/src/c_api/c_api.cc:967: Saving into deprecated binary model format, please consider using `json` or `ubj`. Model format will default to JSON in XGBoost 2.2 if not specified.


In [13]:
res

,IG_model_1,SHAP_model_1,GradientSHAP_model_1,IG_model_2,SHAP_model_2,GradientSHAP_model_2
location,174.995880,4.834416e+00,0.0,0.0,0.007090,0.007090
new_cases_per_million,280641.078385,1.948158e+04,0.0,0.0,0.488640,0.488640
population_cov,86288.595071,6.257131e+04,0.0,0.0,0.043372,0.043372
life_expectancy_cov,45.898819,2.600689e-01,0.0,0.0,0.041766,0.041766
Population,98751.891760,5.985349e+04,0.0,0.0,0.068101,0.068101
Area_km2,443.276121,3.995271e+02,0.0,0.0,0.058819,0.058819
Density_km2,441.777771,1.433053e+02,0.0,0.0,0.027675,0.027675
Year_x,116.521481,0.000000e+00,0.0,0.0,0.000000,0.000000
Meningitis,0.000000,5.040041e-05,0.0,0.0,0.247007,0.247007
Neoplasms,0.242094,3.224433e-03,0.0,0.0,0.009964,0.009964
